In [146]:
import pickle

with open('train.pickle', 'rb') as handle:
     train = pickle.load(handle)
with open('test.pickle', 'rb') as handle:
     test = pickle.load(handle)


In [147]:
#code for the first task, extend the same to the other tasks as well
#adjacent weight sharing

HOPS=3
V = -1 #find the vocabulary size
maxlen = -1 #find the maximum length of a sentence
vocab = {} #find the distinct number of words

In [148]:
#Preprocess the data
import re
punctuation = re.compile(r'[-.?!,":;()|0-9]')
for idx in range(20):
    for key,data in zip(train[idx+1].keys(),train[idx+1].values()):
        memories = data['memory']

        query = data['query'].strip('\n')
        data['query'] = punctuation.sub("", query).strip().lower().split()
        maxlen = max(maxlen,len(data['query']))
        for word in data['query']:
            if word not in vocab:
                vocab[word] = len(vocab)+1

        answer = data['answer'].strip('\n')
        data['answer'] = punctuation.sub("", answer).strip().lower().split()
        maxlen = max(maxlen,len(data['answer']))
        for word in data['answer']:

            if word not in vocab:
                vocab[word] = len(vocab)+1

        data['fact'] = data['fact'].strip('\n').strip()

        for i in range(len(memories)):
            memories[i] = memories[i].strip('\n')
            memories[i] = punctuation.sub("", memories[i])
            memories[i] = memories[i].strip().lower().split()
            maxlen = max(maxlen,len(memories[i]))

            for word in memories[i]:
                if word not in vocab:
                    vocab[word] = len(vocab)+1

        data['memory'] = memories
        train[idx+1][key] = data
    
    

In [149]:
train[1]

{0: {'answer': ['bathroom'],
  'fact': '1',
  'memory': [['mary', 'moved', 'to', 'the', 'bathroom'],
   ['john', 'went', 'to', 'the', 'hallway']],
  'query': ['where', 'is', 'mary']},
 1: {'answer': ['hallway'],
  'fact': '4',
  'memory': [['daniel', 'went', 'back', 'to', 'the', 'hallway'],
   ['sandra', 'moved', 'to', 'the', 'garden']],
  'query': ['where', 'is', 'daniel']},
 2: {'answer': ['hallway'],
  'fact': '4',
  'memory': [['john', 'moved', 'to', 'the', 'office'],
   ['sandra', 'journeyed', 'to', 'the', 'bathroom']],
  'query': ['where', 'is', 'daniel']},
 3: {'answer': ['office'],
  'fact': '11',
  'memory': [['mary', 'moved', 'to', 'the', 'hallway'],
   ['daniel', 'travelled', 'to', 'the', 'office']],
  'query': ['where', 'is', 'daniel']},
 4: {'answer': ['bathroom'],
  'fact': '8',
  'memory': [['john', 'went', 'back', 'to', 'the', 'garden'],
   ['john', 'moved', 'to', 'the', 'bedroom']],
  'query': ['where', 'is', 'sandra']},
 5: {'answer': ['bathroom'],
  'fact': '2',
  'm

In [150]:
#Preprocess the data
import re
punctuation = re.compile(r'[-.?!,":;()|0-9]')
for idx in range(20):
    for key,data in zip(test[idx+1].keys(),test[idx+1].values()):
        memories = data['memory']

        query = data['query'].strip('\n')
        data['query'] = punctuation.sub("", query).strip().lower().split()
        maxlen = max(maxlen,len(data['query']))
        for word in data['query']:
            if word not in vocab:
                vocab[word] = len(vocab)+1

        answer = data['answer'].strip('\n')
        data['answer'] = punctuation.sub("", answer).strip().lower().split()
        maxlen = max(maxlen,len(data['answer']))
        for word in data['answer']:
            if(word == 'milkfootball' ):
                print(query)

            if word not in vocab:
                vocab[word] = len(vocab)+1

        data['fact'] = data['fact'].strip('\n').strip()

        for i in range(len(memories)):
            memories[i] = memories[i].strip('\n')
            memories[i] = punctuation.sub("", memories[i])
            memories[i] = memories[i].strip().lower().split()
            maxlen = max(maxlen,len(memories[i]))

            for word in memories[i]:
                if word not in vocab:
                    vocab[word] = len(vocab)+1

        data['memory'] = memories
        test[idx+1][key] = data
    
    

9 What is Daniel carrying? 
9 What is Mary carrying? 
12 What is Mary carrying? 
14 What is Sandra carrying? 
3 What is Daniel carrying? 
6 What is Daniel carrying? 
14 What is Mary carrying? 
13 What is Mary carrying? 
16 What is Mary carrying? 
9 What is John carrying? 


In [151]:
data['query']

['why', 'did', 'sumit', 'go', 'to', 'the', 'garden']

In [152]:
vocab['<PAD>'] = 0
V = len(vocab)
print(V)
d = 50
H=3
print(maxlen)
RN=True
MEM_SIZE = 50
BATCH_SIZE = 32

175
11


In [153]:
import torch
import torch . nn as nn
import torch . nn . functional as F
import torch.autograd as ag
import numpy as np
# This is our neural networks class
class MemN2N ( nn . Module ):
# Here we define our netwrok structure
    
    def getGaussianMatrix(self,Vo,dim):
        mu, sigma = 0, 0.1 # mean and standard deviation
        s = np.random.normal(mu, sigma, Vo*dim)
        s = s.reshape(Vo,-1)
        s[0] = 0
        return s
    
    def getGaussianMatrix3D(self,x,y,z):
        mu, sigma = 0, 0.1 # mean and standard deviation
        s = np.random.normal(mu, sigma, x*y*z)
        s = s.reshape(x,y,z)
        return s

    def constrainPadding(self):
        self.C0.weight.data[0] = 0
        self.C1.weight.data[0] = 0
        self.C2.weight.data[0] = 0
        self.C3.weight.data[0] = 0

        
    def __init__ ( self, vocab_size = V, embedding_dim = (V,d), hops = H):
        super ( MemN2N,self ). __init__ ()
        self.MEM_SIZE = MEM_SIZE
        self.BATCH_SIZE = BATCH_SIZE
        self.RN = RN
        self.d = d
        self.maxlen = maxlen
        
#         self.memory_size = ag.Variable(torch.zeros(BATCH_SIZE).int().cuda(),requires_grad=False)
        self.L = np.zeros((maxlen,d))
        self.linear_mode = True
        for k in range(self.L.shape[1]-1):
            for j in range(self.L.shape[0]-1):
                self.L[j,k] = (1-float(j+1)/self.L.shape[0]) - ((k+1)/self.L.shape[1])*(1-(2*float(j+1))/self.L.shape[0])
        
        self.L = ag.Variable(torch.from_numpy(self.L).contiguous().cuda(),requires_grad=False)
                
#         self.memories = ag.Variable(torch.zeros(BATCH_SIZE,MEM_SIZE,d).double().cuda(),requires_grad=False)
#         self.output_memories = ag.Variable(torch.zeros(BATCH_SIZE,MEM_SIZE,d).double().cuda(),requires_grad=False)
        self.V = vocab_size
        self.embedding_dim = embedding_dim
        self.hops = hops
        self.C0 = nn . Embedding (embedding_dim[0] , embedding_dim[1], padding_idx=0).double()
        self.C1 = nn . Embedding (embedding_dim[0] , embedding_dim[1], padding_idx=0).double()
        self.C2 = nn . Embedding (embedding_dim[0] , embedding_dim[1], padding_idx=0).double()
        self.C3 = nn . Embedding (embedding_dim[0] , embedding_dim[1], padding_idx=0).double()
        
        #initialize random weight based on Gaussian Distribution
        
        self.C0.weight = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.V,self.d)))
        self.C1.weight = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.V,self.d)))
        self.C2.weight = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.V,self.d)))
        self.C3.weight = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.V,self.d)))

        if torch.cuda.is_available():
            self.T_C0 = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.MEM_SIZE,self.d)).cuda())
            self.T_C1 = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.MEM_SIZE,self.d)).cuda())
            self.T_C2 = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.MEM_SIZE,self.d)).cuda())
            self.T_C3 = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.MEM_SIZE,self.d)).cuda())
        
        else:
            self.T_C0 = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.MEM_SIZE,self.d)))
            self.T_C1 = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.MEM_SIZE,self.d)))
            self.T_C2 = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.MEM_SIZE,self.d)))
            self.T_C3 = nn.Parameter(torch.from_numpy(self.getGaussianMatrix(self.MEM_SIZE,self.d)))
        
        #initialize zero weight to padded input
#         self.constrainPadding()
        
        if torch.cuda.is_available():
            self.C0 = self.C0.cuda()
            self.C1 = self.C1.cuda()
            self.C2 = self.C2.cuda()
            self.C3 = self.C3.cuda()
            self.L = self.L.cuda()
    
    def reset_memory(self):
        self.memories = None
        self.output_memories = None
        
        
    def forward ( self , x , q ):
            
        A_k = self.C0
        B_k = self.C0
        C_k = self.C1
        
        A_k_T = self.T_C0
        C_k_T = self.T_C1
            
        
        mat = self.L.unsqueeze(0).repeat(self.BATCH_SIZE,1,1)
#         mat = self.L.repeat(self.BATCH_SIZE)
        Q = B_k(q)*mat

#         print(weighted_Q.view(-1,d,d))
        Q = torch.sum(Q,dim=1)
        mat1 = self.L.unsqueeze(0).repeat(self.BATCH_SIZE*self.MEM_SIZE,1,1)
        x = x.view(-1,self.maxlen)
        
        for i in range(self.hops):
            self.reset_memory()
            C_k = getattr(self, 'C'+str(i+1))
            C_k_T = getattr(self, 'T_C'+str(i+1))
            
#             print(x.size())
            
#             mat = self.L.repeat(self.BATCH_SIZE,self.MEM_SIZE)
            weighted_M = (A_k(x)*mat1).view(self.BATCH_SIZE,self.MEM_SIZE,self.maxlen,-1)
            weighted_O = (C_k(x)*mat1).view(self.BATCH_SIZE,self.MEM_SIZE,self.maxlen,-1)
            sum_M = torch.sum(weighted_M,dim=2)
            sum_O = torch.sum(weighted_O,dim=2)
            
            self.memories = sum_M + A_k_T.unsqueeze(0).repeat(self.BATCH_SIZE,1,1)
            self.output_memories = sum_O + C_k_T.unsqueeze(0).repeat(self.BATCH_SIZE,1,1)
            
            
#             if(RN == True):
#                 s1 = self.getGaussianMatrix3D(self.BATCH_SIZE,(int(self.MEM_SIZE/10)),self.d)
#                 s2 = self.getGaussianMatrix3D(self.BATCH_SIZE,(int(self.MEM_SIZE/10)),self.d)
#                 if torch.cuda.is_available():
#                     noise1 = ag.Variable(torch.from_numpy(s1).double().cuda())    
#                     noise2 = ag.Variable(torch.from_numpy(s2).double().cuda())    
#                 else:
#                     noise1 = ag.Variable(torch.from_numpy(s1).double())    
#                     noise2 = ag.Variable(torch.from_numpy(s2).double())    
                
#                 self.memories = torch.cat((self.memories,noise1),dim=1)
#                 self.output_memories = torch.cat((self.output_memories,noise2),dim=1)
                
#                 self.MEM_SIZE = int(1.1*self.MEM_SIZE)
            
            product = torch.bmm(Q.unsqueeze(1).repeat(1,self.MEM_SIZE,1).unsqueeze(3).permute(0,1,3,2).view(-1,1,d),self.memories.unsqueeze(3).view(-1,self.d,1)) # batch size x 50
#           print(product.size())
            product = product.view(self.BATCH_SIZE,self.MEM_SIZE,1).squeeze(2)
            if(self.linear_mode == False):
                product = F.softmax(product, dim=1) # batch size x 50
            product = product.unsqueeze(2).repeat(1,1,self.d)
            O_k = torch.sum(product*self.output_memories, dim = 1)
            Q = Q + O_k #batch size x d
            A_k = C_k
            A_k_T = C_k_T
                
#         self.constrainPadding()
        output = torch.mm(Q,C_k.weight.permute(1,0))
    
        return output
                
    # This is a helper function to understand the dimensions
    def num_flat_features ( self , x ):
        size = x.size()[1:] # all dimensions except the batch dimension
        return np.prod(size)

net = MemN2N ()
net.hops = 3
print ( net )
# for name,param in list(net.named_parameters()):
#     if param.requires_grad:
#         print name, param.data
if torch.cuda.is_available():
    net = net.cuda()




MemN2N(
  (C0): Embedding(175, 50, padding_idx=0)
  (C1): Embedding(175, 50, padding_idx=0)
  (C2): Embedding(175, 50, padding_idx=0)
  (C3): Embedding(175, 50, padding_idx=0)
)


In [154]:
vocab['bathroom']

4

In [155]:
criterion = nn . CrossEntropyLoss ()
NUM_SAMPLES = 1000
NUM_BATCHES = (NUM_SAMPLES+BATCH_SIZE-1)/BATCH_SIZE
gamma = 0.01
gamma_i = 0.005
NUM_EPOCHS = 60
optimizer = torch . optim . SGD ( net . parameters () , lr = gamma_i)
scheduler = torch . optim . lr_scheduler . StepLR(optimizer, step_size=NUM_EPOCHS/4, gamma=1)
clip = 40


In [156]:
def strictly_decreasing(L):
    return all(x<=y for x, y in zip(L, L[1:]))

task_perm = np.random.permutation(20)
net.linear_start = False
flag = False
loss_arr = []
val_arr = []
length = dict()
nptasks = dict()
training_data = dict()
validation_data = dict()
train_perm = dict()
valid_perm = dict()
batch_perm = dict()
batch_perm_valid = dict()
for task in task_perm:
    nptasks[task] = np.array(train[task+1].values())
    length[task] = len(nptasks[task]) 
    perm = np.random.permutation(min(NUM_SAMPLES,length[task]))
    train_perm[task] = perm[:int(0.9*min(NUM_SAMPLES,length[task]))]
    valid_perm[task] = perm[int(0.9*min(NUM_SAMPLES,length[task])):]
    training_data[task] = nptasks[task][train_perm[task]]
    validation_data[task] = nptasks[task][valid_perm[task]]

for epoch in range(NUM_EPOCHS):  
    
    for task in task_perm:
        batch_perm[task] = np.random.permutation(len(training_data[task]))
        batch_perm_valid[task] = np.random.permutation(len(validation_data[task]))



    running_loss = 0
    running_valid = 0
    scheduler.step()
    
    for i in range(int(NUM_BATCHES*0.9)):
        
        for task_i in task_perm:
            
                
            batch_perm_t = batch_perm[task_i][BATCH_SIZE*i:BATCH_SIZE*(i+1)]
            training_batch = training_data[task_i][batch_perm_t]
                        
            
            inputs = np.zeros((BATCH_SIZE,MEM_SIZE,maxlen))
            
            queries = np.zeros((BATCH_SIZE,maxlen))
            
            outputs = np.zeros((BATCH_SIZE))
            
            
            
            for batch,batch_i in zip(training_batch,range(BATCH_SIZE)):
                for memory,mem_i in zip(reversed(batch['memory']),range(min(MEM_SIZE,len(batch['memory'])))):
                    for memory_word,word_i in zip(memory,range(len(memory))):
                        inputs[batch_i,mem_i,word_i] = vocab[memory_word]
                for query_word,query_word_i in zip(batch['query'],range(len(batch['query']))):
                        queries[batch_i,query_word_i] = vocab[query_word]
                outputs[batch_i] = vocab[batch['answer'][0]]

                        
            
            if torch.cuda.is_available():
                inputs = ag.Variable(torch.from_numpy(inputs).long().cuda())
                queries = ag.Variable(torch.from_numpy(queries).long().cuda())
                outputs = ag.Variable(torch.from_numpy(outputs).long().cuda())
            else:
                inputs = ag.Variable(torch.from_numpy(inputs).long())
                queries = ag.Variable(torch.from_numpy(queries).long())
                outputs = ag.Variable(torch.from_numpy(outputs).long())
            
                
                
            net.zero_grad()
            op = net(inputs,queries)
            loss = criterion(op,outputs)
            loss.backward()
            for name,param in list(net.named_parameters()):
                if param.requires_grad and name[0]!='T':
                    param.grad[0] = 0
                    

#             for p in net.parameters():
#                 param_norm = p.grad.data.norm(2)
#                 if(param_norm > clip):
#                     p.grad.data.mul_(clip/param_norm)

            nn.utils.clip_grad_norm(net.parameters(), clip, 2)
#             for p in net.parameters():
#                 param_norm = p.grad.data.norm(2)
#                 if(param_norm > clip):
#                     p.grad.data.mul_(clip/param_norm)

                
            optimizer.step()
            
            
            running_loss += loss[0]
            if i % 20 == 19:
                print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss/(20*len(task_perm))))
                                        
    for i in range(int(0.1*NUM_BATCHES)):
        
        for task_i in task_perm:
            
                
            valid_start = BATCH_SIZE*i
            valid_end = BATCH_SIZE*(i+1)
            if(valid_start >= len(batch_perm_valid[task_i])):
                continue
                
            batch_perm_valid_t = batch_perm_valid[task_i][valid_start:valid_end]
            
            validation_batch = validation_data[task_i][batch_perm_valid_t]
            
            
            
            inputs_valid = np.zeros((BATCH_SIZE,MEM_SIZE,maxlen))

            queries_valid = np.zeros((BATCH_SIZE,maxlen))
            
            outputs_valid = np.zeros(BATCH_SIZE)
            
            for batch,batch_i in zip(validation_batch,range(BATCH_SIZE)):
                for memory,mem_i in zip(reversed(batch['memory']),range(min(MEM_SIZE,len(batch['memory'])))):
                    for memory_word,word_i in zip(memory,range(len(memory))):
                        inputs_valid[batch_i,mem_i,word_i] = vocab[memory_word]
                for query_word,query_word_i in zip(batch['query'],range(len(batch['query']))):
                        queries_valid[batch_i,query_word_i] = vocab[query_word]
                outputs_valid[batch_i] = vocab[batch['answer'][0]]
            
            
            if torch.cuda.is_available():
                inputs_valid = ag.Variable(torch.from_numpy(inputs_valid).long().cuda(),volatile = True)
                queries_valid = ag.Variable(torch.from_numpy(queries_valid).long().cuda(),volatile = True)
                outputs_valid = ag.Variable(torch.from_numpy(outputs_valid).long().cuda(),volatile = True)
            else:
                inputs_valid = ag.Variable(torch.from_numpy(inputs_valid).long(),volatile = True)
                queries_valid = ag.Variable(torch.from_numpy(queries_valid).long(),volatile = True)
                outputs_valid = ag.Variable(torch.from_numpy(outputs_valid).long(),volatile = True)

                
                
            op1 = net(inputs_valid,queries_valid)
            loss1 = criterion(op1,outputs_valid)
            
            running_valid += loss1[0]
            print('[%d, %5d] valid loss: %.3f' %
            (epoch + 1, i + 1, running_valid))
            val_arr.append(round(float(running_valid[0].data/(100*len(task_perm))),5))
            running_valid = 0
            if(flag == True):
                if(len(val_arr)>=3 and strictly_decreasing(val_arr[-3:])):
#            and val_arr[len(val_arr)-2]>=val_arr[len(val_arr)-3]) :
                    net.linear_mode = False
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = gamma
                    scheduler.gamma = 1
                    print 'yay'
                    flag = False

        
    


[1,    20] loss: 5.506
[1,    20] loss: 5.518
[1,    20] loss: 5.523
[1,    20] loss: 5.532
[1,    20] loss: 5.544
[1,    20] loss: 5.549
[1,    20] loss: 5.555
[1,    20] loss: 5.560
[1,    20] loss: 5.567
[1,    20] loss: 5.579
[1,    20] loss: 5.582
[1,    20] loss: 5.585
[1,    20] loss: 5.591
[1,    20] loss: 5.599
[1,    20] loss: 5.612
[1,    20] loss: 5.623
[1,    20] loss: 5.628
[1,    20] loss: 5.630
[1,    20] loss: 5.643
[1,    20] loss: 5.656
[1,     1] valid loss: 2.155
[1,     1] valid loss: 3.600
[1,     1] valid loss: 1.164
[1,     1] valid loss: 3.381
[1,     1] valid loss: 5.020
[1,     1] valid loss: 2.021
[1,     1] valid loss: 1.550
[1,     1] valid loss: 1.844
[1,     1] valid loss: 2.547
[1,     1] valid loss: 3.778
[1,     1] valid loss: 0.824
[1,     1] valid loss: 1.673
[1,     1] valid loss: 1.869
[1,     1] valid loss: 3.405
[1,     1] valid loss: 4.353
[1,     1] valid loss: 4.150
[1,     1] valid loss: 1.654
[1,     1] valid loss: 1.240
[1,     1] valid l

[5,    20] loss: 1.665
[5,    20] loss: 1.666
[5,    20] loss: 1.669
[5,    20] loss: 1.672
[5,    20] loss: 1.674
[5,    20] loss: 1.677
[5,    20] loss: 1.680
[5,    20] loss: 1.683
[5,    20] loss: 1.685
[5,    20] loss: 1.698
[5,    20] loss: 1.711
[5,     1] valid loss: 2.050
[5,     1] valid loss: 3.340
[5,     1] valid loss: 0.759
[5,     1] valid loss: 1.937
[5,     1] valid loss: 2.974
[5,     1] valid loss: 0.715
[5,     1] valid loss: 0.788
[5,     1] valid loss: 1.436
[5,     1] valid loss: 2.059
[5,     1] valid loss: 1.159
[5,     1] valid loss: 0.481
[5,     1] valid loss: 1.028
[5,     1] valid loss: 1.089
[5,     1] valid loss: 0.905
[5,     1] valid loss: 1.332
[5,     1] valid loss: 0.798
[5,     1] valid loss: 1.099
[5,     1] valid loss: 0.990
[5,     1] valid loss: 2.454
[5,     1] valid loss: 2.829
[5,     2] valid loss: 1.700
[5,     2] valid loss: 0.876
[5,     2] valid loss: 1.956
[5,     2] valid loss: 2.781
[5,     2] valid loss: 0.675
[5,     2] valid loss:

[9,    20] loss: 1.579
[9,    20] loss: 1.592
[9,     1] valid loss: 1.748
[9,     1] valid loss: 3.339
[9,     1] valid loss: 0.834
[9,     1] valid loss: 1.746
[9,     1] valid loss: 2.801
[9,     1] valid loss: 0.676
[9,     1] valid loss: 0.759
[9,     1] valid loss: 1.250
[9,     1] valid loss: 2.056
[9,     1] valid loss: 1.382
[9,     1] valid loss: 0.231
[9,     1] valid loss: 1.165
[9,     1] valid loss: 1.077
[9,     1] valid loss: 0.711
[9,     1] valid loss: 1.315
[9,     1] valid loss: 0.341
[9,     1] valid loss: 1.103
[9,     1] valid loss: 1.163
[9,     1] valid loss: 2.377
[9,     1] valid loss: 2.482
[9,     2] valid loss: 1.640
[9,     2] valid loss: 0.749
[9,     2] valid loss: 1.976
[9,     2] valid loss: 2.678
[9,     2] valid loss: 0.680
[9,     2] valid loss: 0.707
[9,     2] valid loss: 1.303
[9,     2] valid loss: 1.997
[9,     2] valid loss: 1.003
[9,     2] valid loss: 0.378
[9,     2] valid loss: 1.167
[9,     2] valid loss: 1.182
[9,     2] valid loss: 0.8

[13,    20] loss: 1.529
[13,    20] loss: 1.542
[13,     1] valid loss: 1.714
[13,     1] valid loss: 3.334
[13,     1] valid loss: 0.679
[13,     1] valid loss: 1.901
[13,     1] valid loss: 2.660
[13,     1] valid loss: 0.693
[13,     1] valid loss: 0.677
[13,     1] valid loss: 1.202
[13,     1] valid loss: 2.379
[13,     1] valid loss: 0.963
[13,     1] valid loss: 0.329
[13,     1] valid loss: 1.262
[13,     1] valid loss: 1.181
[13,     1] valid loss: 1.073
[13,     1] valid loss: 1.053
[13,     1] valid loss: 0.166
[13,     1] valid loss: 0.951
[13,     1] valid loss: 1.197
[13,     1] valid loss: 2.302
[13,     1] valid loss: 2.343
[13,     2] valid loss: 1.711
[13,     2] valid loss: 0.769
[13,     2] valid loss: 1.877
[13,     2] valid loss: 2.608
[13,     2] valid loss: 0.658
[13,     2] valid loss: 0.480
[13,     2] valid loss: 1.118
[13,     2] valid loss: 1.965
[13,     2] valid loss: 0.850
[13,     2] valid loss: 0.220
[13,     2] valid loss: 1.106
[13,     2] valid loss

[17,    20] loss: 1.493
[17,    20] loss: 1.505
[17,     1] valid loss: 1.855
[17,     1] valid loss: 3.334
[17,     1] valid loss: 0.699
[17,     1] valid loss: 1.757
[17,     1] valid loss: 2.635
[17,     1] valid loss: 0.712
[17,     1] valid loss: 0.790
[17,     1] valid loss: 1.011
[17,     1] valid loss: 2.126
[17,     1] valid loss: 0.899
[17,     1] valid loss: 0.191
[17,     1] valid loss: 1.085
[17,     1] valid loss: 1.012
[17,     1] valid loss: 0.707
[17,     1] valid loss: 1.189
[17,     1] valid loss: 0.177
[17,     1] valid loss: 0.831
[17,     1] valid loss: 1.145
[17,     1] valid loss: 2.349
[17,     1] valid loss: 2.421
[17,     2] valid loss: 1.776
[17,     2] valid loss: 0.775
[17,     2] valid loss: 1.926
[17,     2] valid loss: 2.723
[17,     2] valid loss: 0.758
[17,     2] valid loss: 0.652
[17,     2] valid loss: 1.045
[17,     2] valid loss: 2.207
[17,     2] valid loss: 1.005
[17,     2] valid loss: 0.464
[17,     2] valid loss: 1.095
[17,     2] valid loss

...

[41,    20] loss: 1.328
[41,    20] loss: 1.340
[41,     1] valid loss: 1.734
[41,     1] valid loss: 3.345
[41,     1] valid loss: 0.577
[41,     1] valid loss: 1.751
[41,     1] valid loss: 2.559
[41,     1] valid loss: 0.739
[41,     1] valid loss: 0.106
[41,     1] valid loss: 1.039
[41,     1] valid loss: 2.756
[41,     1] valid loss: 1.168
[41,     1] valid loss: 0.143
[41,     1] valid loss: 1.098
[41,     1] valid loss: 0.908
[41,     1] valid loss: 0.660
[41,     1] valid loss: 1.500
[41,     1] valid loss: 0.115
[41,     1] valid loss: 0.838
[41,     1] valid loss: 0.781
[41,     1] valid loss: 2.236
[41,     1] valid loss: 2.284
[41,     2] valid loss: 1.649
[41,     2] valid loss: 0.670
[41,     2] valid loss: 1.646
[41,     2] valid loss: 2.637
[41,     2] valid loss: 0.711
[41,     2] valid loss: 0.157
[41,     2] valid loss: 0.893
[41,     2] valid loss: 2.875
[41,     2] valid loss: 0.871
[41,     2] valid loss: 0.144
[41,     2] valid loss: 0.899
[41,     2] valid loss

[45,    20] loss: 1.324
[45,    20] loss: 1.337
[45,     1] valid loss: 1.591
[45,     1] valid loss: 3.350
[45,     1] valid loss: 0.603
[45,     1] valid loss: 1.398
[45,     1] valid loss: 2.630
[45,     1] valid loss: 0.749
[45,     1] valid loss: 0.329
[45,     1] valid loss: 0.659
[45,     1] valid loss: 2.638
[45,     1] valid loss: 1.071
[45,     1] valid loss: 0.097
[45,     1] valid loss: 0.905
[45,     1] valid loss: 1.031
[45,     1] valid loss: 0.658
[45,     1] valid loss: 1.222
[45,     1] valid loss: 0.119
[45,     1] valid loss: 0.490
[45,     1] valid loss: 0.704
[45,     1] valid loss: 2.231
[45,     1] valid loss: 2.196
[45,     2] valid loss: 1.993
[45,     2] valid loss: 0.570
[45,     2] valid loss: 1.947
[45,     2] valid loss: 2.520
[45,     2] valid loss: 0.779
[45,     2] valid loss: 0.123
[45,     2] valid loss: 0.863
[45,     2] valid loss: 1.903
[45,     2] valid loss: 0.879
[45,     2] valid loss: 0.086
[45,     2] valid loss: 1.315
[45,     2] valid loss

[49,    20] loss: 1.302
[49,    20] loss: 1.315
[49,     1] valid loss: 1.575
[49,     1] valid loss: 3.356
[49,     1] valid loss: 0.669
[49,     1] valid loss: 1.480
[49,     1] valid loss: 2.691
[49,     1] valid loss: 0.749
[49,     1] valid loss: 0.201
[49,     1] valid loss: 0.714
[49,     1] valid loss: 2.497
[49,     1] valid loss: 0.757
[49,     1] valid loss: 0.165
[49,     1] valid loss: 1.211
[49,     1] valid loss: 1.021
[49,     1] valid loss: 0.600
[49,     1] valid loss: 1.137
[49,     1] valid loss: 0.179
[49,     1] valid loss: 0.471
[49,     1] valid loss: 0.706
[49,     1] valid loss: 2.358
[49,     1] valid loss: 2.243
[49,     2] valid loss: 1.884
[49,     2] valid loss: 0.494
[49,     2] valid loss: 1.443
[49,     2] valid loss: 2.573
[49,     2] valid loss: 0.807
[49,     2] valid loss: 0.052
[49,     2] valid loss: 0.826
[49,     2] valid loss: 3.218
[49,     2] valid loss: 0.871
[49,     2] valid loss: 0.126
[49,     2] valid loss: 1.216
[49,     2] valid loss

[53,    20] loss: 1.297
[53,    20] loss: 1.310
[53,     1] valid loss: 1.648
[53,     1] valid loss: 3.357
[53,     1] valid loss: 0.547
[53,     1] valid loss: 1.503
[53,     1] valid loss: 2.624
[53,     1] valid loss: 0.699
[53,     1] valid loss: 0.124
[53,     1] valid loss: 1.070
[53,     1] valid loss: 2.554
[53,     1] valid loss: 1.170
[53,     1] valid loss: 0.189
[53,     1] valid loss: 1.126
[53,     1] valid loss: 0.833
[53,     1] valid loss: 0.713
[53,     1] valid loss: 0.994
[53,     1] valid loss: 0.047
[53,     1] valid loss: 0.598
[53,     1] valid loss: 0.806
[53,     1] valid loss: 2.300
[53,     1] valid loss: 2.186
[53,     2] valid loss: 1.792
[53,     2] valid loss: 0.634
[53,     2] valid loss: 1.445
[53,     2] valid loss: 2.624
[53,     2] valid loss: 0.748
[53,     2] valid loss: 0.161
[53,     2] valid loss: 0.856
[53,     2] valid loss: 3.226
[53,     2] valid loss: 0.845
[53,     2] valid loss: 0.096
[53,     2] valid loss: 1.068
[53,     2] valid loss

[57,    20] loss: 1.288
[57,    20] loss: 1.301
[57,     1] valid loss: 1.665
[57,     1] valid loss: 3.361
[57,     1] valid loss: 0.644
[57,     1] valid loss: 1.832
[57,     1] valid loss: 2.599
[57,     1] valid loss: 0.759
[57,     1] valid loss: 0.166
[57,     1] valid loss: 0.740
[57,     1] valid loss: 3.367
[57,     1] valid loss: 1.036
[57,     1] valid loss: 0.110
[57,     1] valid loss: 1.167
[57,     1] valid loss: 1.097
[57,     1] valid loss: 0.803
[57,     1] valid loss: 1.123
[57,     1] valid loss: 0.114
[57,     1] valid loss: 0.318
[57,     1] valid loss: 0.571
[57,     1] valid loss: 2.286
[57,     1] valid loss: 2.182
[57,     2] valid loss: 2.002
[57,     2] valid loss: 0.514
[57,     2] valid loss: 1.322
[57,     2] valid loss: 2.536
[57,     2] valid loss: 0.748
[57,     2] valid loss: 0.132
[57,     2] valid loss: 0.993
[57,     2] valid loss: 2.359
[57,     2] valid loss: 0.916
[57,     2] valid loss: 0.152
[57,     2] valid loss: 1.008
[57,     2] valid loss

In [166]:
accuracy = []
task = 9
testing_data = np.array(test[task+1].values())
length = len(testing_data) 
        
for i in range(min(NUM_BATCHES,(length+BATCH_SIZE-1)/BATCH_SIZE)):
    testing_batch = testing_data[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
#             print(testing_batch)
            
    inputs_test = np.zeros((BATCH_SIZE,MEM_SIZE,maxlen))
            
            
    queries_test = np.zeros((BATCH_SIZE,maxlen))
            
    outputs_test = np.zeros((BATCH_SIZE))
            
            
            
    for batch,batch_i in zip(testing_batch,range(BATCH_SIZE)):
        for memory,mem_i in zip(reversed(batch['memory']),range(min(MEM_SIZE,len(batch['memory'])))):
            for memory_word,word_i in zip(memory,range(len(memory))):
                inputs_test[batch_i,mem_i,word_i] = vocab[memory_word]
        for query_word,query_word_i in zip(batch['query'],range(len(batch['query']))):
            queries_test[batch_i,query_word_i] = vocab[query_word]
        outputs_test[batch_i] = vocab[batch['answer'][0]]
#     print(inputs_test)
#     print(i)
#     print(outputs_test)
            
    if torch.cuda.is_available():
        inputs_test = ag.Variable(torch.from_numpy(inputs_test).long().cuda(),volatile = True)
        queries_test = ag.Variable(torch.from_numpy(queries_test).long().cuda(),volatile = True)
        outputs_test = ag.Variable(torch.from_numpy(outputs_test).long().cuda(),volatile = True)
    else:
        inputs_test = ag.Variable(torch.from_numpy(inputs_test).long(),volatile = True)
        queries_test = ag.Variable(torch.from_numpy(queries_test).long(),volatile = True)
        outputs_test = ag.Variable(torch.from_numpy(outputs_test).long(),volatile = True)
                
                
                
    op_test = net(inputs_test,queries_test)
    acc = np.mean(np.argmax(op_test.data.cpu().numpy(),axis=1) == outputs_test.data.cpu().numpy())
            
    accuracy.append(acc)
print(np.mean(accuracy))
            


0.537109375


In [17]:
for value,key in zip(vocab.values(),vocab.keys()):
    if(len(key) > 10):
        print(value)
        print(key)

68
milkfootball
69
milkfootballapple
71
applemilkfootball
174
footballapplemilk
64
footballapple
67
footballmilk
72
applefootballmilk
66
applefootball


[2, 3]


In [148]:
criterion = nn . CrossEntropyLoss ()
NUM_SAMPLES = 1000
NUM_BATCHES = (NUM_SAMPLES+BATCH_SIZE-1)/BATCH_SIZE
gamma = 0.01
gamma_i = 0.005
NUM_EPOCHS = 60
optimizer = torch . optim . SGD ( net . parameters () , lr = gamma_i)
scheduler = torch . optim . lr_scheduler . StepLR(optimizer, step_size=NUM_EPOCHS/4, gamma=1)
clip = 40
MEM_SIZE = 50


In [53]:

task_perm = [2]#np.random.permutation(20)
loss_arr = []
val_arr = []
flag = True
for epoch in range(NUM_EPOCHS):  
    running_loss = 0
    running_valid = 0
    scheduler.step()
    for task in task_perm:
        nptasks = np.array(train[task+1].values())
        length = len(nptasks) 
        perm = np.random.permutation(min(NUM_SAMPLES,length))
        train_perm = perm[:int(0.9*min(NUM_SAMPLES,length))]
        valid_perm = perm[int(0.9*min(NUM_SAMPLES,length)):]
        
        training_data = nptasks[train_perm]
        validation_data = nptasks[valid_perm]
        for i in range(min(NUM_BATCHES,(length+BATCH_SIZE-1)/BATCH_SIZE)):
            
            batch_perm = np.random.permutation(len(training_data))
            batch_perm_valid = np.random.permutation(len(validation_data))
            
            batch_perm = batch_perm[:BATCH_SIZE]
            batch_perm_valid = batch_perm_valid[:BATCH_SIZE]
            
            training_batch = training_data[batch_perm]
            validation_batch = validation_data[batch_perm_valid]
            
            x = [len(batch['memory']) for batch in training_batch]
            maxmem = np.max(x)
            
            x = [len(batch['memory']) for batch in validation_batch]
            maxmem_valid = np.max(x)
            
            inputs = np.zeros((BATCH_SIZE,maxmem,maxlen))
            inputs_valid = np.zeros((BATCH_SIZE,maxmem_valid,maxlen))

            queries = np.zeros((BATCH_SIZE,maxlen))
            queries_valid = np.zeros((BATCH_SIZE,maxlen))
            
            outputs = np.zeros((BATCH_SIZE))
            outputs_valid = np.zeros((BATCH_SIZE))
            
            
            
            for batch,batch_i in zip(training_batch,range(BATCH_SIZE)):
                for memory,mem_i in zip(reversed(batch['memory']),range(len(batch['memory']))):
                    for memory_word,word_i in zip(memory,range(len(memory))):
                        inputs[batch_i,mem_i,word_i] = vocab[memory_word]
                for query_word,query_word_i in zip(batch['query'],range(len(batch['query']))):
                        queries[batch_i,query_word_i] = vocab[query_word]
                outputs[batch_i] = vocab[query_word]
            
            for batch,batch_i in zip(validation_batch,range(BATCH_SIZE)):
                for memory,mem_i in zip(reversed(batch['memory']),range(len(batch['memory']))):
                    for memory_word,word_i in zip(memory,range(len(memory))):
                        inputs_valid[batch_i,mem_i,word_i] = vocab[memory_word]
                for query_word,query_word_i in zip(batch['query'],range(len(batch['query']))):
                        queries_valid[batch_i,query_word_i] = vocab[query_word]
                outputs_valid[batch_i] = vocab[query_word]
            
            
            if torch.cuda.is_available():
                inputs = ag.Variable(torch.from_numpy(inputs).long().cuda())
                queries = ag.Variable(torch.from_numpy(queries).long().cuda())
                outputs = ag.Variable(torch.from_numpy(outputs).long().cuda())
                inputs_valid = ag.Variable(torch.from_numpy(inputs_valid).long().cuda(),volatile = True)
                queries_valid = ag.Variable(torch.from_numpy(queries_valid).long().cuda(),volatile = True)
                outputs_valid = ag.Variable(torch.from_numpy(outputs_valid).long().cuda(),volatile = True)
            else:
                inputs = ag.Variable(torch.from_numpy(inputs).long())
                queries = ag.Variable(torch.from_numpy(queries).long())
                outputs = ag.Variable(torch.from_numpy(outputs).long())
                inputs_valid = ag.Variable(torch.from_numpy(inputs_valid).long(),volatile = True)
                queries_valid = ag.Variable(torch.from_numpy(queries_valid).long(),volatile = True)
                outputs_valid = ag.Variable(torch.from_numpy(outputs_valid).long(),volatile = True)

                
                
            net.zero_grad()
            net.reset_memory()
            op = net(inputs,queries)
            loss = criterion(op,outputs)
            loss.backward()
            nn.utils.clip_grad_norm(net.parameters(), clip, 2)
            optimizer.step()
            
            op1 = net(inputs_valid,queries_valid)
            loss1 = criterion(op1,outputs_valid)
            
            running_valid += loss1[0]
            running_loss += loss[0]
            if i % 20 == 19:
                print('[%d, %5d] loss: %.3f %.3f' %
                (epoch + 1, i + 1, running_loss,running_valid))
                loss_arr.append(float(running_loss[0].data))
                val_arr.append(float(running_valid[0].data))
                running_loss = 0
                running_valid = 0
                if(flag == True):
                    if(len(val_arr) >= 10 and val_arr[len(val_arr)-1]>=val_arr[len(val_arr)-2] and val_arr[len(val_arr)-2]>=val_arr[len(val_arr)-3] \
                    and val_arr[len(val_arr)-3]>=val_arr[len(val_arr)-4]):
                        net.linear_mode = False
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = gamma
                        scheduler.gamma = 0.5
                        print 'yay'
                        flag = False
                                        
        
        
    


RuntimeError: The size of tensor a (122) must match the size of tensor b (50) at non-singleton dimension 3